In [1]:
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 13.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints


from neuralforecast import NeuralForecast
from neuralforecast.models import TFT
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.losses.pytorch import MSE, MAE, MQLoss, DistributionLoss

In [3]:
from numpy.random import seed
from random import randrange

In [4]:
df = pd.read_excel("SP100_vol.xlsx", index_col="Date",parse_dates=True)
columns=df.columns
df_static=pd.read_excel("Static_df.xlsx")
n_series=len(df.columns)
train = df.iloc[:int(len(df['NKE'])*0.7)]
validation_length=int(len(df['NKE'])*0.7)-int(len(df['NKE'])*0.5)
train=train.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [5]:
n_inputs = [3,5,10,15,21,42,84]
hidden_size = [50,100,150,200,300,500]
epochs=[50,100,150,250,350,450,550,650,750]
dropout=[0,0.2,0.3,0.4]
n_head=[2,4,5,8,10,20]
attn_dropout=[0,0.2,0.3,0.4]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
losses=[MSE(),MAE(),MQLoss(level=[80, 90]),DistributionLoss(distribution='StudentT', level=[80, 90])]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']

# Test

In [6]:
test_length=int(len(df['NKE'])*0.3)
df1=df.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [7]:
RMSE = []
QLIKE = []
MAE = []
MAPE = []
for m in range (5):
  randomseed=randrange(129228148)
  model=TFT(h=1, input_size=15, stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=500,
       n_head=20, attn_dropout=0.4, dropout=0.2, loss=MSE(),
       max_steps=350, learning_rate=learning_rate[3],
       num_lr_decays=num_lr_decays[3],
       step_size=1,
       scaler_type='minmax',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=df1,val_size=1,static_df=df_static,n_windows=None, test_size=test_length-test_length%1,step_size=1)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE.append(np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true)))
  QLIKE.append((np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true)))
  MAE.append(np.sum(abs(Y_true-Y_hat))/len(Y_true))
  MAPE.append(np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true))

INFO:lightning_fabric.utilities.seed:Seed set to 55894500


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

In [8]:
for m in range (5):
  randomseed=randrange(129228148)
  model=TFT(h=1, input_size=15, stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=500,
       n_head=20, attn_dropout=0.4, dropout=0.2, loss=MSE(),
       max_steps=350, learning_rate=learning_rate[3],
       num_lr_decays=num_lr_decays[3],
       step_size=1,
       scaler_type='minmax',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=df1,val_size=1,static_df=df_static,n_windows=None, test_size=test_length-test_length%1,step_size=1)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE.append(np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true)))
  QLIKE.append((np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true)))
  MAE.append(np.sum(abs(Y_true-Y_hat))/len(Y_true))
  MAPE.append(np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true))

INFO:lightning_fabric.utilities.seed:Seed set to 1341232


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

TypeError: ignored

In [10]:
for m in range (1):
  randomseed=randrange(129228148)
  model=TFT(h=1, input_size=15, stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=500,
       n_head=20, attn_dropout=0.4, dropout=0.2, loss=MSE(),
       max_steps=350, learning_rate=learning_rate[3],
       num_lr_decays=num_lr_decays[3],
       step_size=1,
       scaler_type='minmax',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=df1,val_size=1,static_df=df_static,n_windows=None, test_size=test_length-test_length%1,step_size=1)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE.append(np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true)))
  QLIKE.append((np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true)))
  MAE.append(np.sum(abs(Y_true-Y_hat))/len(Y_true))
  MAPE.append(np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true))

INFO:lightning_fabric.utilities.seed:Seed set to 129226031


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
for m in range (5):
  randomseed=randrange(129228148)
  model=TFT(h=1, input_size=15, stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=500,
       n_head=20, attn_dropout=0.4, dropout=0.2, loss=MSE(),
       max_steps=350, learning_rate=learning_rate[3],
       num_lr_decays=num_lr_decays[3],
       step_size=1,
       scaler_type='minmax',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=df1,val_size=1,static_df=df_static,n_windows=None, test_size=test_length-test_length%1,step_size=1)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE.append(np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true)))
  QLIKE.append((np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true)))
  MAE.append(np.sum(abs(Y_true-Y_hat))/len(Y_true))
  MAPE.append(np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true))

INFO:lightning_fabric.utilities.seed:Seed set to 52253045


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
for m in range (5):
  randomseed=randrange(129228148)
  model=TFT(h=1, input_size=15, stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=500,
       n_head=20, attn_dropout=0.4, dropout=0.2, loss=MSE(),
       max_steps=350, learning_rate=learning_rate[3],
       num_lr_decays=num_lr_decays[3],
       step_size=1,
       scaler_type='minmax',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=df1,val_size=1,static_df=df_static,n_windows=None, test_size=test_length-test_length%1,step_size=1)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE.append(np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true)))
  QLIKE.append((np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true)))
  MAE.append(np.sum(abs(Y_true-Y_hat))/len(Y_true))
  MAPE.append(np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true))

INFO:lightning_fabric.utilities.seed:Seed set to 77291329


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

In [19]:
print(f"RMSE mean: {np.mean(RMSE)} | QLIKE mean: {np.mean(QLIKE)} | MAE mean: {np.mean(MAE)} | MAPE mean: {np.mean(MAPE)}")

RMSE mean: 0.003966431259917005 | QLIKE mean: 0.031225139635549626 | MAE mean: 0.0025211762243882536 | MAPE mean: 0.19720794968483152


In [20]:
print(f"RMSE std: {np.std(RMSE)} | QLIKE std: {np.std(QLIKE)} | MAE std: {np.std(MAE)} | MAPE std: {np.std(MAPE)}")

RMSE std: 4.38790746277139e-05 | QLIKE std: 0.0009871270098867355 | MAE std: 7.425611448908206e-05 | MAPE std: 0.009985490591574255


In [21]:
from scipy.stats import anderson
print(f"RMSE: {anderson(RMSE)}")
print(f"QLIKE: {anderson(QLIKE)}")
print(f"MAE: {anderson(MAE)}")
print(f"MAPE: {anderson(MAPE)}")

RMSE: AndersonResult(statistic=1.007874915012458, critical_values=array([0.506, 0.577, 0.692, 0.807, 0.96 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.003966431259917005, scale=4.50189806775634e-05)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
QLIKE: AndersonResult(statistic=1.803411878325953, critical_values=array([0.506, 0.577, 0.692, 0.807, 0.96 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.031225139635549626, scale=0.0010127709429023385)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
MAE: AndersonResult(statistic=0.7323213541240321, critical_values=array([0.506, 0.577, 0.692, 0.807, 0.96 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.0025211762243882536, scale=7.618516597575495e-05)
 success: True
 message: '`anderson` successfully fit the distrib

In [22]:
df_errors = pd.DataFrame({
    'RMSE': RMSE,
    'QLIKE': QLIKE,
    'MAE': MAE,
    'MAPE': MAPE
})

df_errors.to_excel("TFT forecasts.xlsx")